# Laboratoire 6
## Filtre de Kalman Étendu (EKF)

### Partie 1 - Calibration du capteur

In [ ]:
# Importation des modules
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path
import numpy as np
import math
import time
from matplotlib import gridspec
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('nbagg')
from robmob.robot import Robot
from robmob.sensors import SharpSensor
from IPython.display import clear_output

Modifiez la valeur de `ip_robot` selon le robot qui vous a été assigné.

In [ ]:
# Connexion au robot
ip_robot = '192.168.0.111'
robot = Robot(ip_robot)
robot.connect()

In [ ]:
sharp_sensor_id = 0 # 0 ou 1 selon votre robot
sharp_sensor = SharpSensor(sharp_sensor_id)
robot.add_sensor(sharp_sensor)

Utilisez cette cellule pour vérifier que vous avez bien choisis le bon capeut IR. Pour ce laboratoire on utilise le capteur Infra

In [ ]:
data = sharp_sensor.read_buffer()
print(data.shape)
print("La lecture moyenne est de %0.2f volts" % np.average(data))

#### Calibration
Prenez une série de mesures séparées de 5 cm, entre 30 cm et 120 cm. Prenez note que le buffer prend environ 2 secondes à ce remplir, donc prenez votre temps entre les mesurese.
Soyez attentifs, car ces mesures
serviront à calibrer votre capteur infrarouge et aussi à tester votre filtre de Kalman.

Dans la cellule suivante, utilisez ```t``` (take) pour prendre la mesure et ```r``` (retry) pour reprendre la mesure. 

In [ ]:
xs = np.array([x for x in range(30, 125, 5)]) * 0.01 #Distances à calibrer en mètres
zs = []

In [ ]:
# Capture des données de calibration

for x in xs:
    key_in = ''
    while key_in != 't':
        buf = sharp_sensor.read_buffer()
        z = np.mean(buf)
        print("mesure pour x = %f" % x)
        print("moyenne de %d mesures: %f" % (len(buf), z))
        print("(t)ake / (r)etry")
        key_in = input()
    zs.append(z)
    clear_output()
    time.sleep(0.5)

zs = np.asarray(zs)

print(xs.shape, zs.shape)

In [ ]:
# Sauvegarde des zs de calibration sur le disque (au cas où vous redémarriez le notebook)
np.savetxt("./lab6_calib.txt", np.vstack((xs, zs)))

In [ ]:
xs, zs = np.loadtxt("./lab6_calib.txt")
print(xs.shape, zs.shape)

In [ ]:
def calibrate(xs, zs):
    """
    Args:
      - xs: distances en mètres des mesures zs
      - zs: mesures en Volts
    Return:
      - Fonction linéaire qui prend en entrée l'inverse de la distance en mètres et retourne une mesure en Volts 
        (c'est la fonction de capteur)
      - Les coefficients de K_1 et K_2 de la fonction linéaire de capteur
    """
    xs_inverse_meters = 1.0 / xs
    coeffs = np.polyfit(xs_inverse_meters, zs, deg=1)
    sensor_function = np.poly1d(coeffs)
    return sensor_function, coeffs

In [ ]:
sensor_fn, coeffs = calibrate(xs, zs)
k_1_sharp = coeffs[1]
k_2_sharp = coeffs[0]
print("K_1: %f" % k_1_sharp)
print("K_2: %f" % k_2_sharp)

#Plot calibration
xs_inv = 1.0 / xs
plt.scatter(xs_inv, zs)
plt.plot(xs_inv, sensor_fn(xs_inv))
plt.xlabel("1/x [1/m]")
plt.ylabel("z (V)")


![kalman-ir](./img/kalman-ir.png)


Comme illustré à l’image ci-haut, le capteur infrarouge fonctionne essentiellement avec l’inverse de la
distance. Il sera donc approximé par la fonction suivante :

$$
f_{infrarouge}(x) = K_{1 Sharp} + \frac{K_{2 Sharp}}{x}, x > 30
$$

Comme cette fonction est
bijective, on peut trouver un estimé de distance $x$ directement à partir de la mesure infrarouge $z$, avec
l’inverse de la fonction :

$$
f^{-1}_{infrarouge}(z) = \frac{K_{2 Sharp}}{z - K_{1 Sharp}}
$$

Vous aurez besoin de cette fonction pour initialiser l’estimé $X$ d’un filtre quand vous ne connaissez pas
la position de départ du système. Elle sera aussi utilisée par le script pour comparer directement les
mesures du capteur infrarouge avec vos estimés. **Notez que ces équations (1) et (2) ne sont valides que
pour $x$ > 0.30 m : votre robot ne pourra donc jamais être à moins de 0.30 m du mur**.

### Partie 2 - Modèle du système
Comme estimé de bruit sur le capteur infrarouge, nous allons prendre la valeur de $\sigma^2_{infrarouge} = (0.02V)^2$, ce qui correspond approximativement à un palier du convertisseur analogique-numérique. Le bruit de
déplacement sera $\sigma^2_{pas} = (0.002 m)^2$, ce qui donne essentiellement un écart-type de 2 mm sur un pas du
robot. Fait à noter, le choix de ces valeurs influencera grandement le comportement du filtre. Ainsi, si
vous choisissez un estimé de bruit de capteur $\sigma^2_{infrarouge}$
plus petit, vous indiquez au filtre qu’il doit
augmenter sa confiance envers les mesures du capteur. De la même manière, si vous diminuez la valeur
estimée du bruit sur les commandes $\sigma^2_{pas}$, il augmentera sa confiance envers l’estimé $x(k+1\,|\,k)$ obtenu lors
de la phase de prédiction. Nous y reviendrons lors des manipulations.

> **NOTE** Le bruit associé à la conversion analogique-numérique est généralement modélisé comme une distribution uniforme entre
±un demi-palier de conversion. Pour les curieux, une discussion complète de ce sujet est disponible sur wikipedia :
http://en.wikipedia.org/wiki/Quantization_error

La dynamique du système et la commande sont linéaires pour ce système:

$$
x(k + 1\,|\,k) = x(k) + u(k)
$$

Ce qui nous donne les matrices ```F``` (ou $\Phi$ sur les acétates) $= [1]$ et ```G``` (ou $\Gamma$ sur les acétates) $= [1]$. La fonction de mesure, telle qu'exprimée précédemment, est quant à elle non-linéaire:

$$
f_{infrarouge}(x) = K_{1 Sharp} + \frac{K_{2 Sharp}}{x}, x > 30
$$

Pour l’utiliser dans un filtre de Kalman, nous allons donc devoir linéariser cette fonction autour du point
d’opération $x$ (la position estimée du robot). La jacobienne ```H``` (ou $\Lambda$ sur les acétates) est utilisé par le
filtre de Kalman étendu (EKF) pour effectuer cette linéarisation :

$$
H = \Big[\frac{\delta}{\delta x} f_{infrarouge}(x)\Big] = \Big[ \frac{-K_{2Sharp}}{x^2}\Big]
$$

Cette matrice ```H``` (ou $\Lambda$ sur les acétates) est de taille $n\times m$, où $n$ est le nombre de capteurs et $m$ la longueur
du vecteur d’état. Pour ce système, sa taille est de 1x1, car nous avons une variable d’état (la position),
et un seul capteur (infrarouge Sharp). 

> **IMPORTANT!** La valeur numérique de ```H``` est recalculée à chaque
itération par le filtre EKF, car la pente de la fonction du capteur dépend de la position $x$. Aussi, le filtre
utilise l’estimé ```X``` pour la calculer, car nous ne connaissons pas la vraie valeur de $x$. C’est d’ailleurs une
des sources d’échec du filtre : si ```X``` et $x$ sont très différents, alors la valeur de ```H``` sera incorrecte, et le filtre
risque de diverger. Plus la fonction sera non-linéaire, et plus la distance |```X```-$x$| posera problème.

### Partie 3 - Collecte de données

Il est possible d’utiliser un filtre de manière offline, c’est-à-dire après que les données soient toutes
capturées par le robot. Ceci a l’avantage de faciliter votre familiarisation avec ce filtre.

Dans cette partie, nous allons collecter des données que nous allons ensuite utiliser dans la partie suivante sur le filtre de Kalman.

Placez votre robot à la position de départ 0.30 m. Le script suivant fait avancer le robot à coups de 0.05m en enregistrant à chaque fois la mesure de l'infrarouge.

In [ ]:
delta_t = 1       # intervalle des mesures en sec
speed = 0.05      # m/s

x_0 = 0.3
x_final = 1.20
distance = (x_final - x_0)
n_steps = math.ceil(distance / speed / delta_t)
step_distance = delta_t * speed

positions = np.zeros((n_steps,))
measures = np.zeros((n_steps,))

for i in range(n_steps):
    positions[i] = x_0 + i * step_distance
    measures[i] = sharp_sensor.peek_data()
    
    
    robot.linear_movement_precise(step_distance, speed)
    time.sleep(0.6) #Laisser le temps au buffer du Sharp de se remplir

print(np.vstack((positions, measures)))

Si vous êtes satisfait des données collectées, utilisez la cellule suivante pour les enregistrer

In [ ]:
data_to_save = np.vstack((positions, measures))
np.savetxt("./lab6_offline_data.txt", data_to_save)

### Partie 4 - Filtre


In [ ]:
# Load data
positions, measures = np.loadtxt("./lab6_offline_data.txt")

In [ ]:
delta_t = 1          # intervalle des mesures en sec
n_steps = positions.shape[0]
speed = 0.05         # m/s
sigma_sharp = 0.02   # (V) ecart-type sur voltage du capteur infrarouge
sigma_kobuki = 0.006 # (m) ecart-type sur un pas du robot kobuki

Cv = sigma_kobuki ** 2
Cw = sigma_sharp ** 2

# Actions du système
u = 0.05

# Calibration - doit avoir été faites plus haut
assert(k_1_sharp is not None and k_2_sharp is not None)


X = positions[0]  # On connait exactement la position initiale du robot
P = 0.0

#Données à recueillir pour tracer les graphiques
graph_data = {
    "x": [positions[0]],
    "x_estimate": [X],
    "z": [measures[0]],
    "K": [0],
    "P": [P],
    "error": [positions[0] - X],
    "time": [0]
}

for i in range(1, n_steps): # On commence le compteur à 1, car 0 est l'état initial à 30cm, et 1 est après
                            # la première commande
    time = i * delta_t
    
    z = measures[i]
    
    F = np.array([1])       # Phi, pour dynamique systeme
    G = np.array([1])       # Gamma, pour matrice de commande
    
    # Propagation de l'estimé
    X = F * X + G * u
    P = F * P * F.T + G * Cv * G.T  # Propagation covariance

    
    # ========= Calcul des matrices Jacobiennes pour mise-a-jour =============
    H = -k_2_sharp / X ** 2  # Je dois utiliser mon estimé pour ici, car je ne connais pas la vraie valeur

    z_hat =  k_1_sharp + k_2_sharp / X;

    # ======== Mise-a-jour ========
    K = P * H.T / (H * P * H.T + Cw)   # Gain Kalman
    r = (z - z_hat)                    # Innovation
    X = X + K*r
    P = (1 - K * H) * P
    
    
    # Cueillette des données pour le graphique
    graph_data["x"].append(positions[i])
    graph_data["x_estimate"].append(X)
    graph_data["z"].append(z)
    graph_data["K"].append(K)
    graph_data["P"].append(P)
    graph_data["error"].append(positions[i] - X)
    graph_data["time"].append(time)
    
    

### Partie 5 - Visualisation des résultats

In [ ]:
fig = plt.figure(figsize=(6, 12))
gs = gridspec.GridSpec(3, 1)

ax1 = plt.subplot(gs[0])
ax1.set_title("Position (commandes)")
ax1.plot(graph_data['time'], graph_data['x'], '-o', label="Commandes")
ax1.grid(True)

ax2 = plt.subplot(gs[1])
ax2.set_title("Estimé de position (filtre)")
ax2.plot(graph_data['time'], graph_data['x_estimate'], '-o', label="Estimé Filtre")
ax2.set_ylabel('position (m)')
ax2.grid(True)

# On inverse la fonction de capteur pour trouver la position estimée
# par le capteur infrarouge
z_numpy = np.array(graph_data['z'])
x_sharp_estimated = k_2_sharp / (z_numpy - k_1_sharp)
ax3 = plt.subplot(gs[2])
ax3.set_title("Estimé de position (sharp)")
ax3.plot(graph_data['time'], x_sharp_estimated, '-o', label="Estimé Sharp")
ax3.set_xlabel("temps (s)")
ax3.grid(True)

plt.show()

In [ ]:
plt.title("Position et estimées")
plt.xlabel("temps (s)")
plt.ylabel("position (m)")
plt.plot(graph_data['time'], graph_data['x'], label="Commandes")
plt.scatter(graph_data['time'], graph_data['x_estimate'], marker='x', label="Estimé Filtre")
plt.scatter(graph_data['time'], x_sharp_estimated, marker='o', color='red', label="Estimé Sharp")
plt.legend(loc='upper left')

In [ ]:
plt.title("Erreur en fonction du temps")
plt.xlabel("temps (s)")
plt.ylabel("erreur (m)")
plt.scatter(graph_data['time'], graph_data['error'])

In [ ]:
cov = np.array(graph_data['P'])*1000000
plt.title("Covariance en fonction du temps")
plt.xlabel("temps (s)")
plt.ylabel("Covariance ((mm)^2)")
plt.scatter(graph_data['time'], cov)

In [ ]:
plt.title("Gain de Kalman en fonction du temps")
plt.xlabel("temps (s)")
plt.ylabel("Gain")
plt.scatter(graph_data['time'], graph_data['K'])

In [ ]:
sharp_errors = graph_data['x'] - x_sharp_estimated
plt.title("Écart-type infrarouge=%.3f mm" % np.std(sharp_errors * 1000))
plt.hist(sharp_errors, bins=12)
plt.show()

In [ ]:
errors = np.asarray(graph_data['error'])
plt.title("Écart-type filtre Kalman=%.3f mm" % np.std(errors * 1000))
plt.hist(errors, bins=10)
plt.show()

#### Discussion

Quel comportement voyez-vous? En particulier, vous devriez observer que 

- la précision sur la position estimée diminue au fil du temps, ce qui reflète l’accumulation progressive des erreurs;

- la matrice de covariance P reflète bien cette situation, en accroissant avec le temps;
- la valeur absolue 2 du gain K du filtre augmente au début, car le filtre doit tenir compte de plus en plus de la mesure pour corriger l’estimé de la position à mesure que notre estimé perd de la précision.


Normalement, la valeur de la covariance P et du gain K se stabilise, ce qui n’est pas le cas en ce
moment. Pourquoi? Eh bien à mesure que vous vous éloignez, votre capteur perd énormément de
précision, et la matrice P et le gain K doivent refléter cette situation. Pour un scénario plus réaliste, le
robot resterait toujours dans les mêmes parages, et P et K convergeraient vers une valeur fixe. Rappelez-
vous que pour ce système :

$$
K = \frac{PH^T}{HPH^T + \sigma^2_{infrarouge}}
$$


> **NOTE** Pour ce système, le gain K est négatif. La courbe montera donc une valeur descendante pour une augmentation de la valeur absolue du gain.

### Partie 6 - Trajet inverse
Répétez l'expérience précédente mais en faisant comme si le robot était parti loin du mur pour s'en approcher. Vous pouvez utiliser les données que vous avez déjà enregistrées pour faire cette expérience.

Voici comment inverser un array numpy:

In [ ]:
a = np.array([1, 2, 3, 4, 5])
a_inv = a[::-1]
print(a, a_inv)
print(type(a_inv))

Observez les changements sur les matrices P et K, en fonction du temps.

### Partie 7 - Position initiale inconnue
Pour ce cas, nous allons assumer que votre robot ne connaisse pas sa position au début. Pour
refléter cette situation, changer les valeurs initiales de $X$ et $P$:

$$
X = f^{-1}_{infrarouge}(z)
$$

$$
P = H^{-1} \sigma^2_{infrarouge}(H^T)^{-1}
$$

La valeur de $P$ indique au filtre que la précision de l’estimé est égale à celle du capteur. Pourquoi pré- et
post-multiplier par l’inverse de H? Parce que $\sigma^2_{infrarouge}$
est défini dans l’espace des mesures du capteur (en
Volt), et qu’il faut donc estimer quelle est la précision de l’estimé de la position (en m). Cette opération
va faire cette conversion pour nous.

Enlever les modifications précédentes (celles qui faisaient tourner les données en sens inverse), et
exécuter le filtre à nouveau. Quel comportement voyez-vous? En particulier, vous devriez observer que :
la matrice de covariance $P$ diminue après la première itération, car on a acquis de l’information.

### Partie 8 - Erreurs biaisées
Si la valeur moyenne des pas de votre robot ne correspond pas à $u$ (dans le scripte de la partie 4), ceci correspond à la situation
où le bruit de déplacement est biaisé, ce qui viole les hypothèses d’utilisation de ce type de filtre. Cette
situation est très néfaste pour le filtre, qui aura tendance à avoir une erreur beaucoup plus élevée que
normalement. 
Pour ce test, utilisez les données du fichier de calibration à la place des données acquises automatiquement, car celle-ci sont sans biais. Autrement dit, utilisez:

In [ ]:
positions, measures = np.loadtxt("./lab6_calib.txt")

En modifiant légèrement la valeur de $u$ dans le début du code du filtre (par exemple le faire passer de
0.05 à 0.0475), regardez comment le filtre accumule une erreur importante : vous verrez les cercles verts
qui s’éloignent progressivement de la valeur réelle. À la lumière de cette expérience, croyez-vous qu’il
est important de ne pas avoir un système contenant une erreur de commande biaisé? De plus, croyez-vous que les données acquisent automatiquement étaient biaisées?

### Partie 9 - Évaluation de la variance (optionnel)

Faites 8 déplacements de 10 pas de 0.05 m, en utilisant le script de la partie 3. Partez toujours du
même endroit. Mesurez la position finale du robot avec la règle, et notez-là. Calculez l’erreur sur la
position finale pour les 8 séries. Quelle est la variance sur cette erreur? Est-elle comparable à la variance
estimée par le filtre?